In [19]:
import pyspark
import os

In [2]:
sc = pyspark.SparkContext(appName="Pi")

In [3]:
students = sc.textFile("students.csv")

In [4]:
students.collect()

['Name, Marks', 'Pulkit, 99', 'Aman, 100', 'Jhon, 89', 'Jerry, 78', 'Rob, 69']

In [8]:
filterStudents = students.filter(lambda row : 'a' in row)

In [9]:
filterStudents.collect()

['Name, Marks', 'Aman, 100']

In [17]:
streaming_file = os.path.join(os.path.pardir, "real-time-streaming", 'streaming-demo.py')

In [39]:
%%writefile $streaming_file

import sys

from pyspark import SparkContext
from pyspark.streaming import StreamingContext

if __name__ == "__main__":
    
    sc = SparkContext(appName= "StreamingData")
    ssc = StreamingContext(sc, 2)
    
    ssc.checkpoint(os.path.join(os.path.pardir, "real-time-streaming", 'tmp'))
    
    lines = ssc.socketTextStream(sys.argv[1], int(sys.argv[2]))
    
    counts = lines.flatMap(lambda line: line.split(" "))\
                    .filter(lambda word: "ERROR" in word)\
                    .map(lambda word: (word, 1))\
                    .reduceByKey(lambda a, b: a+b)
    
    counts.pprint()
    
    ssc.start()
    ssc.awaitTermination()
    

Writing $streaming_file


In [ ]:
!spark-submit streaming-demo.py localhost 9991

In [41]:
twitter_streaming_file = os.path.join(os.path.pardir, "real-time-streaming", 'testtwitterstream-message.py')

In [46]:
%%writefile $twitter_streaming_file


import sys

import tweepy
from tweepy import OAuthHandler, Stream
from tweepy.streaming import StreamListener

class TweetsListener(StreamListener):
    
    def __init__(self):
        print('Tweet Listener class initialized.')
    
    def on_data(self, data):
        print("on_data: {0}".format(data))
        return True
    
    def on_error(self, status):
        print("error: {0}".format(status))
        return True


if __name__ == "__main__":

    api_key = "05vmjOLTZazXj2doCsLYPXMgx"
    api_secret = "OxV7iOFKA3x3nBBF9WHSpQ0w1SVcNdEjOt3GZPmFqDszahb31L"
    
    access_token = "1427298992-FNne49AJM0fnq6ZiHBB7RcjcxsslFwLD7Uj4Pnj"
    access_token_secret = "7V4RDqhgxaemcb42mbBIY50T6eSYDesN6TzMcrxZKohww"
    
    auth = OAuthHandler(api_key, api_secret)
    auth.set_access_token(access_token, access_token_secret)
    
    twitter_stream = Stream(auth, TweetsListener())
    twitter_stream.filter(track=["#"])

Overwriting ../real-time-streaming/testtwitterstream-message.py


In [34]:
twitter_streaming_live_file = os.path.join(os.path.pardir, "real-time-streaming", 'twitterstreaming.py')

In [40]:
%%writefile $twitter_streaming_live_file

import os

from pyspark import SparkContext
from pyspark.streaming import StreamingContext

if __name__ == "__main__":
    
    sc = SparkContext(appName= "StreamingData")
    ssc = StreamingContext(sc, 2)
    
    ssc.checkpoint(os.path.join(os.path.pardir, "real-time-streaming", 'tmp'))
    
    lines = ssc.socketTextStream("localhost", 7777)
    
    def count_words(newValues, lastSum):
        if lastSum is None:
            lastSum = 0
        return sum(newValues, lastSum)
    
    words_counts = lines.flatMap(lambda line: line.split(" "))\
                    .filter(lambda word: "ERROR" in word)\
                    .map(lambda word: (word, 1))\
                    .updateStateByKey(count_words)
    
    words_counts.pprint()
    
    ssc.start()
    ssc.awaitTermination()

Overwriting ../real-time-streaming/twitterstreaming.py
